In [ ]:
# Setup chunk to install and load required packages
knitr::opts_chunk$set(warning = FALSE, message = FALSE)
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'glmnet',
               'randomForest', 'xgboost','patchwork',
               'paletteer', 'here', 'doParallel', 'summarytools')


## **Regression - Experimenting with more powerful regression models**

In the previous notebook, we used simple regression models to look at the relationship between features of a bike rentals dataset. In this notebook, we'll experiment with more complex models to improve our regression performance.

Let's begin by loading the bicycle sharing data as a tibble and viewing the first few rows. We'll also split our data into training and test datasets.


In [ ]:
# Load the required packages and make them available in your current R session
suppressPackageStartupMessages({
  library(tidyverse)
  library(tidymodels)
  library(lubridate)
  library(paletteer)
})

# Import the data into the R session
bike_data <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/data/daily-bike-share.csv", show_col_types = FALSE)

# Parse dates then extract days
bike_data <- bike_data %>%
  mutate(dteday = mdy(dteday)) %>% 
  mutate(day = day(dteday))

# Select desired features and labels
bike_select <- bike_data %>% 
  select(c(season, mnth, holiday, weekday, workingday, weathersit,
           temp, atemp, hum, windspeed, rentals)) %>% 
  mutate(across(1:6, factor))

# Split 70% of the data for training and the rest for tesing
set.seed(2056)
bike_split <- bike_select %>% 
  initial_split(prop = 0.7,
  # splitting data evenly on the holiday variable
                strata = holiday)

# Extract the data in each split
bike_train <- training(bike_split)
bike_test <- testing(bike_split)

# Specify multiple regression metrics
eval_metrics <- metric_set(rmse, rsq)


cat("Training Set", nrow(bike_train), "rows",
    "\nTest Set", nrow(bike_test), "rows")


This results into the following two datasets:

-   *bike_train*: subset of the dataset used to train the model.

-   *bike_test*: subset of the dataset used to validate the model.

Now we're ready to train a model by fitting a suitable regression algorithm to the training data.

### Experiment with Algorithms

The linear regression algorithm we used last time to train the model has some predictive capability, but there are many kinds of regression algorithm we could try, including:

-   **Linear algorithms**: Not just the Linear Regression algorithm we used above (which is technically an *Ordinary Least Squares* algorithm), but other variants such as *Lasso* and *Ridge*.
-   **Tree-based algorithms**: Algorithms that build a decision tree to reach a prediction.
-   **Ensemble algorithms**: Algorithms that combine the outputs of multiple base algorithms to improve generalizability.

> **Note**: For a full list of parsnip model types and engines, see parsnip [model types and engines](https://www.tidymodels.org/find/parsnip/#models) and explore corresponding [model arguments](https://www.tidymodels.org/find/parsnip/#model-args) too.

### Try Another Linear Algorithm

Let's try training our regression model by using a `Lasso` (**Least absolute shrinkage and selection operator**) algorithm. In Tidymodels, we can do this easily by just changing the model specification then the rest is a breeze!

Here we'll set up one model specification for lasso regression; we picked a value for `penalty` (sort of randomly) and we set `mixture = 1` to specifiy a lasso model (When mixture = 1, it is a pure lasso model).

We'll also make a model specification in a more succinct way than we did last time!


In [ ]:
# Build a lasso model specification
lasso_spec <- linear_reg(
  engine = "glmnet",
  mode = "regression",
  penalty = 1,
  mixture = 1)

# Train a lasso regression model
lasso_mod <- lasso_spec %>% 
  fit(rentals ~ ., data = bike_train)

# Make predictions for test data
results <- bike_test %>% 
  bind_cols(lasso_mod %>% predict(new_data = bike_test) %>% 
              rename(predictions = .pred))

# Evaluate the model
lasso_metrics <- eval_metrics(data = results,
                                    truth = rentals,
                                    estimate = predictions) 


# Plot predicted vs actual
theme_set(theme_light())
lasso_plt <- results %>% 
  ggplot(mapping = aes(x = rentals, y = predictions)) +
  geom_point(size = 1.6, color = 'darkorchid') +
  # overlay regression line
  geom_smooth(method = 'lm', color = 'black', se = F) +
  ggtitle("Daily Bike Share Predictions") +
  xlab("Actual Labels") +
  ylab("Predicted Labels") +
  theme(plot.title = element_text(hjust = 0.5))

# Return evaluations
list(lasso_metrics, lasso_plt)
  


🤔 Hmmmm... Not much of an improvement. We could improve the performance metrics by estimating the right regularization hyperparameter `penalty`. This can be figured out by using `resampling` and `tuning` the model which we'll discuss in just a few.

### Try a Decision Tree Algorithm

As an alternative to a linear model, there's a category of algorithms for machine learning that uses a `tree-based` approach in which the features in the dataset are examined in a series of evaluations, each of which results in a *branch* in a *decision tree* based on the feature value. At the end of each series of branches are leaf-nodes with the predicted label value based on the feature values.\
\
It's easiest to see how this works with an example. Let's train a Decision Tree regression model using the bike rental data. After training the model, the code below will print the model definition and a text representation of the tree it uses to predict label values.


In [ ]:
# Build a decision tree specification
tree_spec <- decision_tree(
  engine = "rpart",
  mode = "regression")

# Train a decision tree model 
tree_mod <- tree_spec %>% 
  fit(rentals ~ ., data = bike_train)

# Print model
tree_mod


So now we have a tree-based model; but is it any good? Let's evaluate it with the test data. We'll also try out a new function: `augmet()`

> `augmnet()` allows you to make and add model predictions to the given data


In [ ]:
# Make and bind predictions to test data a
results <- tree_mod %>% 
  augment(new_data = bike_test) %>% 
  rename(predictions = .pred)

# Evaluate the model
tree_metrics <- eval_metrics(data = results,
                                  truth = rentals,
                                  estimate = predictions)

# Plot predicted vs actual
tree_plt <- results %>% 
  ggplot(mapping = aes(x = rentals, y = predictions)) +
  geom_point(color = 'tomato') +
  # overlay regression line
  geom_smooth(method = 'lm', color = 'steelblue', se = F) +
  ggtitle("Daily Bike Share Predictions") +
  xlab("Actual Labels") +
  ylab("Predicted Labels") +
  theme(plot.title = element_text(hjust = 0.5))

# Return evaluations
list(tree_metrics, tree_plt)


The tree-based model doesn't seem to have much improvement over the linear model. We can also see that it's predicting constant values for a given range of predictors. So, what else could we try?

### Try an Ensemble Algorithm

Ensemble algorithms work by combining multiple base estimators to produce an optimal model, either by applying an *aggregate function* to a collection of base models (sometimes referred to a `bagging`) or by building a sequence of models that build on one another to improve predictive performance (referred to as `boosting`).

For example, let's try a Random Forest model, which applies an averaging function to multiple Decision Tree models for a better overall model.


In [ ]:
# For reproducibility
set.seed(2056)

# Build a random forest model specification
rf_spec <- rand_forest() %>% 
  set_engine('randomForest') %>% 
  set_mode('regression')

# Train a random forest model 
rf_mod <- rf_spec %>% 
  fit(rentals ~ ., data = bike_train)

# Print model
rf_mod


So now we have a random forest model; but is it any good? Let's evaluate it with the test data.



In [ ]:
# Make and bind predictions to test data a
results <- rf_mod %>% 
  augment(new_data = bike_test) %>% 
  rename(predictions = .pred)


# Evaluate the model
rf_metrics <- eval_metrics(data = results,
                                  truth = rentals,
                                  estimate = predictions)


# Plot predicted vs actual
rf_plt <- results %>% 
  ggplot(mapping = aes(x = rentals, y = predictions)) +
  geom_point(color = '#6CBE50FF') +
  # overlay regression line
  geom_smooth(method = 'lm', color = '#2B7FF9FF', se = F) +
  ggtitle("Daily Bike Share Predictions") +
  xlab("Actual Labels") +
  ylab("Predicted Labels") +
  theme(plot.title = element_text(hjust = 0.5))

# Return evaluations
list(rf_metrics, rf_plt)


🤩 Whoa! That's a step in the right direction.

Let's also try a *boosting* ensemble algorithm. We'll use a `Gradient Boosting` estimator, which like a Random Forest algorithm builds multiple trees, but instead of building them all independently and taking the average result, each tree is `built` on the `outputs` of the `previous one` in an attempt to incrementally reduce the *loss* (error) in the model.

In this tutorial, we'll demonstrate how to implement *Gradient Boosting Machines* using the **xgboost** engine.


In [ ]:
# For reproducibility
set.seed(2056)

# Build an xgboost model specification
boost_spec <- boost_tree() %>% 
  set_engine('xgboost') %>% 
  set_mode('regression')

# Train an xgboost model 
boost_mod <- boost_spec %>% 
  fit(rentals ~ ., data = bike_train)


# Print model
boost_mod


From the output, we actually see that Gradient Boosting Machines combine a series of base models, each of which is created sequentially and depends on the previous models, in an attempt to incrementally reduce the error in the model.

So now we have an XGboost model; but is it any good🤷? Again, let's evaluate it with the test data.


In [ ]:
# Make and bind predictions to test data a
results <- boost_mod %>% 
  augment(new_data = bike_test) %>% 
  rename(predictions = .pred)

# Evaluate the model
boost_metrics <- eval_metrics(data = results,
                                  truth = rentals,
                                  estimate = predictions) 

# Plot predicted vs actual
boost_plt <- results %>% 
  ggplot(mapping = aes(x = rentals, y = predictions)) +
  geom_point(color = '#4D3161FF') +
  # overlay regression line
  geom_smooth(method = 'lm', color = 'black', se = F) +
  ggtitle("Daily Bike Share Predictions") +
  xlab("Actual Labels") +
  ylab("Predicted Labels") +
  theme(plot.title = element_text(hjust = 0.5))

# Return evaluations
list(boost_metrics, boost_plt)


Okay not bad👌! We are definitely getting somewhere. Can we do better?

### **Summary**

Here we've tried a number of new regression algorithms to improve performance. In our notebook we'll look at `tuning` these algorithms to improve performance. Next, we'll take a look at `Data Preprocessing` and `model hyperparameters`.

### **Further Reading**

To learn more about Tidymodels, see the [Tidymodels documentation](https://www.tidymodels.org/).
